## Better Prediction with GradientBoost Regressor

In [65]:
import pandas as pd
import numpy as np
import scipy.stats as stats
from datetime import datetime
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.metrics import mean_squared_error,mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge,Lasso,RidgeCV,LassoCV,ElasticNet,ElasticNetCV,LinearRegression
from sklearn.model_selection import train_test_split,GridSearchCV
sns.set()
import pylab
from sklearn.tree import DecisionTreeClassifier,export_graphviz
from IPython.display import Image
from sklearn.metrics import accuracy_score, confusion_matrix,roc_curve,roc_auc_score
from sklearn.metrics import f1_score as f1
from sklearn import tree
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier,GradientBoostingClassifier,GradientBoostingRegressor
from catboost import CatBoostClassifier,CatBoostRegressor
from lightgbm import LGBMClassifier,LGBMRegressor
from xgboost import XGBClassifier,XGBRegressor
from sklearn.ensemble import AdaBoostRegressor

In [2]:
df=pd.read_csv("C:\\Users\\yashw\\Downloads\\car data.csv")

In [3]:
df

,Car_Name,Year,Selling_Price,Present_Price,Kms_Driven,Fuel_Type,Seller_Type,Transmission,Owner
0,ritz,2014,3.35,5.59,27000,Petrol,Dealer,Manual,0
1,sx4,2013,4.75,9.54,43000,Diesel,Dealer,Manual,0
2,ciaz,2017,7.25,9.85,6900,Petrol,Dealer,Manual,0
3,wagon r,2011,2.85,4.15,5200,Petrol,Dealer,Manual,0
4,swift,2014,4.60,6.87,42450,Diesel,Dealer,Manual,0
...,...,...,...,...,...,...,...,...,...
296,city,2016,9.50,11.60,33988,Diesel,Dealer,Manual,0
297,brio,2015,4.00,5.90,60000,Petrol,Dealer,Manual,0
298,city,2009,3.35,11.00,87934,Petrol,Dealer,Manual,0
299,city,2017,11.50,12.50,9000,Diesel,Dealer,Manual,0


In [4]:
df['Car_Age']=2022-df['Year']

In [8]:
df['Car_Age'].nunique()

16

In [10]:
df.isnull().sum()

Car_Name         0
Year             0
Selling_Price    0
Present_Price    0
Kms_Driven       0
Fuel_Type        0
Seller_Type      0
Transmission     0
Owner            0
Car_Age          0
dtype: int64

In [12]:
df=df.drop(columns='Car_Name')

In [13]:
df

,Year,Selling_Price,Present_Price,Kms_Driven,Fuel_Type,Seller_Type,Transmission,Owner,Car_Age
0,2014,3.35,5.59,27000,Petrol,Dealer,Manual,0,8
1,2013,4.75,9.54,43000,Diesel,Dealer,Manual,0,9
2,2017,7.25,9.85,6900,Petrol,Dealer,Manual,0,5
3,2011,2.85,4.15,5200,Petrol,Dealer,Manual,0,11
4,2014,4.60,6.87,42450,Diesel,Dealer,Manual,0,8
...,...,...,...,...,...,...,...,...,...
296,2016,9.50,11.60,33988,Diesel,Dealer,Manual,0,6
297,2015,4.00,5.90,60000,Petrol,Dealer,Manual,0,7
298,2009,3.35,11.00,87934,Petrol,Dealer,Manual,0,13
299,2017,11.50,12.50,9000,Diesel,Dealer,Manual,0,5


In [14]:
df=pd.get_dummies(df,columns=['Fuel_Type','Seller_Type','Transmission'])

In [15]:
df

,Year,Selling_Price,Present_Price,Kms_Driven,Owner,Car_Age,Fuel_Type_CNG,Fuel_Type_Diesel,Fuel_Type_Petrol,Seller_Type_Dealer,Seller_Type_Individual,Transmission_Automatic,Transmission_Manual
0,2014,3.35,5.59,27000,0,8,0,0,1,1,0,0,1
1,2013,4.75,9.54,43000,0,9,0,1,0,1,0,0,1
2,2017,7.25,9.85,6900,0,5,0,0,1,1,0,0,1
3,2011,2.85,4.15,5200,0,11,0,0,1,1,0,0,1
4,2014,4.60,6.87,42450,0,8,0,1,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
296,2016,9.50,11.60,33988,0,6,0,1,0,1,0,0,1
297,2015,4.00,5.90,60000,0,7,0,0,1,1,0,0,1
298,2009,3.35,11.00,87934,0,13,0,0,1,1,0,0,1
299,2017,11.50,12.50,9000,0,5,0,1,0,1,0,0,1


In [25]:
X=df.drop(columns='Selling_Price')
Y=df['Selling_Price']

In [26]:
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.20,random_state=4)

## Applying AdaBoost Regressor

In [27]:
lm=AdaBoostRegressor()
lm.fit(x_train,y_train)

AdaBoostRegressor()

In [28]:
y_pred=lm.predict(x_test)

In [29]:
y_pred

array([ 4.53131579,  6.24545455,  4.08017241,  5.33714286,  5.05285714,
        1.00464286,  1.00464286,  1.00464286,  5.33714286,  3.26111111,
        1.29482759,  3.73181818,  6.43478261,  6.39025316,  5.33714286,
        1.00464286, 10.936375  ,  3.26111111,  5.05285714,  7.19742574,
       20.58      ,  1.44642857,  1.00464286,  1.29482759,  7.43953271,
        6.39025316,  3.73181818,  1.00464286,  7.87452632,  4.82105769,
        1.00464286,  5.05285714,  1.00464286,  1.00464286,  6.43478261,
        6.8128    ,  1.00464286,  5.27304348,  7.23041237,  7.0862069 ,
        3.26111111, 10.72735294,  4.49819048,  1.00464286,  1.00464286,
        1.00464286,  1.00464286,  7.48887097,  4.3       ,  5.50863636,
       10.936375  ,  4.2455102 ,  7.19742574,  5.33714286,  1.29482759,
        3.18      ,  3.98703704,  1.00464286,  7.19742574, 20.58      ,
        3.80714286])

In [31]:
lm.score(x_test,y_test)

0.8719629308205024

In [32]:
def adj_r2(x,y):
    r2=lm.score(x,y)
    n = x.shape[0]
    p = x.shape[1]
    adjusted_r2 = 1-(1-r2)*(n-1)/(n-p-1)
    return adjusted_r2

In [33]:
adj_r2(x_test,y_test)

0.839953663525628

In [46]:
print('MSE: ',mean_squared_error(y_test,y_pred))
print('MAPE: ',mean_absolute_percentage_error(y_test, y_pred))
print('MAE: ',mean_absolute_error(y_test,y_pred))

MSE:  2.4735718203998305
MAPE:  0.13769481002391892
MAE:  0.6939679909399057


## Applying GradientBoost Regressor

In [39]:
lm=GradientBoostingRegressor()
lm.fit(x_train,y_train)

GradientBoostingRegressor()

In [40]:
y_pred=lm.predict(x_test)

In [41]:
y_pred

array([ 4.15323488,  6.54245681,  3.49531111,  5.59837935,  4.61525402,
        0.2945156 ,  0.44270974,  0.2945156 ,  5.43814976,  2.77764028,
        1.18503858,  2.9789813 ,  5.9960832 ,  6.21317882,  5.31530987,
        0.2319324 , 10.46324996,  2.62124942,  4.42278336,  6.85864298,
       21.51182917,  1.39194889,  0.51971885,  1.06371824,  8.53579786,
        6.21317882,  2.61600343,  0.55861803,  7.88525835,  4.5579483 ,
        0.63566344,  4.72523741,  0.23842318,  0.5799327 ,  6.64723171,
        5.32873739,  0.56469003,  3.41586931,  7.5360357 , 10.21988874,
        2.56541199,  9.54719634,  3.98677349,  0.52211357,  0.65666958,
        0.75901583,  0.2126003 ,  5.74942765,  3.95946145,  5.63719604,
       12.04275105,  3.00240892,  7.34870454,  5.44303533,  1.12562015,
        3.04096098,  3.53828813,  0.53006437,  7.50053737, 22.38475947,
        1.61437644])

In [42]:
lm.score(x_test,y_test)

0.9185233185517606

In [43]:
def adj_r2(x,y):
    r2=lm.score(x,y)
    n = x.shape[0]
    p = x.shape[1]
    adjusted_r2 = 1-(1-r2)*(n-1)/(n-p-1)
    return adjusted_r2

In [44]:
adj_r2(x_test,y_test)

0.8981541481897007

In [45]:
print('MSE: ',mean_squared_error(y_test,y_pred))
print('MAPE: ',mean_absolute_percentage_error(y_test, y_pred))
print('MAE: ',mean_absolute_error(y_test,y_pred))

MSE:  2.4735718203998305
MAPE:  0.13769481002391892
MAE:  0.6939679909399057


## Aplying XGBoost Regressor

In [48]:
lm=XGBRegressor()
lm.fit(x_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, ...)

In [49]:
y_pred=lm.predict(x_test)

In [50]:
y_pred

array([ 4.388774  ,  6.118173  ,  3.172683  ,  5.67301   ,  4.027845  ,
        0.24560346,  0.4176978 ,  0.21866083,  5.7898917 ,  2.8472874 ,
        1.0873415 ,  2.6739342 ,  5.1711893 ,  5.5524974 ,  5.167827  ,
        0.10748269, 10.957262  ,  2.5373623 ,  4.3731737 ,  6.5755286 ,
       21.175884  ,  1.2911918 ,  0.31455094,  1.0266016 ,  9.0978985 ,
        5.73515   ,  2.7720337 ,  0.46840072,  8.49724   ,  4.8989887 ,
        0.5757851 ,  5.090941  ,  0.3752539 ,  0.511618  ,  6.6226735 ,
        4.4587836 ,  0.50065446,  3.2820828 ,  8.137753  ,  9.283548  ,
        2.3375897 ,  9.271842  ,  3.421968  ,  0.6044202 ,  0.4005244 ,
        0.7353603 ,  0.49741712,  4.9778504 ,  4.4702797 ,  5.9879923 ,
       11.995163  ,  3.221559  ,  7.74464   ,  5.491209  ,  1.0397716 ,
        3.2584279 ,  3.4054372 ,  0.4248927 ,  7.2729383 , 20.964842  ,
        1.1778178 ], dtype=float32)

In [51]:
lm.score(x_test,y_test)

0.8980274023334114

In [52]:
def adj_r2(x,y):
    r2=lm.score(x,y)
    n = x.shape[0]
    p = x.shape[1]
    adjusted_r2 = 1-(1-r2)*(n-1)/(n-p-1)
    return adjusted_r2

In [53]:
adj_r2(x_test,y_test)

0.8725342529167642

In [55]:
print('MSE: ',mean_squared_error(y_test,y_pred))
print('MAPE: ',mean_absolute_percentage_error(y_test, y_pred))
print('MAE: ',mean_absolute_error(y_test,y_pred))

MSE:  3.0958126860049444
MAPE:  0.1625146790330753
MAE:  0.7559122396687992


## Applying CatBoost Regressor

In [57]:
lm=CatBoostRegressor()
lm.fit(x_train,y_train)

Learning rate set to 0.032678
0:	learn: 4.8511369	total: 150ms	remaining: 2m 29s
1:	learn: 4.7512486	total: 154ms	remaining: 1m 16s
2:	learn: 4.6542130	total: 159ms	remaining: 52.8s
3:	learn: 4.5651944	total: 163ms	remaining: 40.7s
4:	learn: 4.4830564	total: 169ms	remaining: 33.6s
5:	learn: 4.3926437	total: 174ms	remaining: 28.9s
6:	learn: 4.3042192	total: 179ms	remaining: 25.4s
7:	learn: 4.2332224	total: 183ms	remaining: 22.7s
8:	learn: 4.1451373	total: 187ms	remaining: 20.6s
9:	learn: 4.0672133	total: 191ms	remaining: 18.9s
10:	learn: 3.9850518	total: 194ms	remaining: 17.4s
11:	learn: 3.9141361	total: 197ms	remaining: 16.2s
12:	learn: 3.8514825	total: 199ms	remaining: 15.1s
13:	learn: 3.7965853	total: 201ms	remaining: 14.2s
14:	learn: 3.7260545	total: 204ms	remaining: 13.4s
15:	learn: 3.6602260	total: 208ms	remaining: 12.8s
16:	learn: 3.5879905	total: 211ms	remaining: 12.2s
17:	learn: 3.5238620	total: 213ms	remaining: 11.6s
18:	learn: 3.4724306	total: 216ms	remaining: 11.1s
19:	learn

In [58]:
y_pred=lm.predict(x_test)

In [59]:
y_pred

array([ 4.26575911,  6.36262086,  3.33568243,  5.64116888,  4.26844215,
        0.2980363 ,  0.39578337,  0.28203898,  5.465125  ,  3.21599207,
        0.81074098,  3.14467169,  6.10394246,  6.76199175,  4.53144794,
        0.22135094, 11.19416021,  2.51539268,  4.17402368,  6.87686628,
       21.99619689,  1.45397445,  0.68021146,  0.90939753,  8.93954961,
        6.78640166,  2.28475396,  0.59587072,  8.76658614,  5.13308662,
        0.62772647,  4.63692562,  1.18526513,  0.55187988,  7.61816666,
        4.37773308,  0.5627757 ,  2.85040232,  7.63516145,  9.65513871,
        3.00221229,  9.42300531,  3.89677018,  0.58112452,  0.59086848,
        0.7569439 ,  0.42942585,  5.46194252,  4.13459407,  5.80154618,
       11.71931095,  2.8726116 ,  7.77634664,  6.40262438,  0.95239504,
        2.82510003,  3.31947737,  0.70919337,  7.65006889, 19.10025824,
        1.82067305])

In [60]:
lm.score(x_test,y_test)

0.8779471073846821

In [61]:
def adj_r2(x,y):
    r2=lm.score(x,y)
    n = x.shape[0]
    p = x.shape[1]
    adjusted_r2 = 1-(1-r2)*(n-1)/(n-p-1)
    return adjusted_r2

In [62]:
adj_r2(x_test,y_test)

0.8474338842308526

In [64]:
print('MSE: ',mean_squared_error(y_test,y_pred))
print('MAPE: ',mean_absolute_percentage_error(y_test, y_pred))
print('MAE: ',mean_absolute_error(y_test,y_pred))

MSE:  3.7054355970957507
MAPE:  0.2079216371834841
MAE:  0.7104587914010019


## Applying LightBoost Regressor

In [68]:
lm=LGBMRegressor()
lm.fit(x_train,y_train)

LGBMRegressor()

In [69]:
y_pred=lm.predict(x_test)

In [70]:
y_pred

array([ 4.41207424,  5.95085049,  3.25657052,  5.40982915,  4.18976364,
        0.11820056,  0.28217525,  0.23414563,  5.14722213,  2.98552738,
        1.14433538,  3.04628454,  5.34387209,  6.04875901,  4.86371118,
       -0.15571207, 15.41148426,  2.57127219,  3.89506671,  6.73945635,
       18.20844781,  1.33556316,  1.05993264,  1.12980941,  8.65920334,
        6.00573081,  1.81627685,  0.65992423, 13.1411604 ,  5.06093557,
        0.54849428,  4.80812113,  0.26808669,  1.221823  ,  7.12802446,
        2.8764296 ,  0.57736449,  1.40123495,  7.67468295, 15.82480724,
        2.09876756, 10.91688333,  3.35543688,  0.66878077,  0.50042684,
        0.67664958,  0.72017468,  9.33286245,  4.97937907,  6.18359419,
       15.86884014,  2.10525638,  8.66495949,  5.52842701,  0.97523928,
        2.55672656,  3.03628981,  0.46244039,  6.51904484, 20.66157839,
        1.38057284])

In [72]:
lm.score(x_test,y_test)

0.8411370365473742

In [73]:
def adj_r2(x,y):
    r2=lm.score(x,y)
    n = x.shape[0]
    p = x.shape[1]
    adjusted_r2 = 1-(1-r2)*(n-1)/(n-p-1)
    return adjusted_r2

In [74]:
adj_r2(x_test,y_test)

0.8014212956842177

In [75]:
print('MSE: ',mean_squared_error(y_test,y_pred))
print('MAPE: ',mean_absolute_percentage_error(y_test, y_pred))
print('MAE: ',mean_absolute_error(y_test,y_pred))

MSE:  4.82296213734801
MAPE:  0.30160909892112714
MAE:  1.0759922087214495


In [82]:
d=['e','E','as','w']
c=[]
for i in d:
    if(i.isupper()==True):
        c.append(i.lower())
    else:
        c.append((i.upper()))
        
        

In [83]:
c

['E', 'e', 'AS', 'W']